In [8]:
import cudaq
from cudaq import spin

In [ ]:
from RBM_surrogate import *
#from New_MCMC_Proposal import *
from get_conn import get_conn
from Sampling import *

In [ ]:
import os
import numpy as np
#import netket as nk
import scipy as sp
import numpy.linalg as la
import scipy.linalg as spla
import time
import matplotlib.pyplot as plt
import json

In [ ]:
N=10
M=10
D=0
beta=1

In [11]:
H = spin.x(1)

In [ ]:
cudaq.SpinOperator()

In [3]:
def string_to_ops(str):
    Pauli_Ops = []
    Coeffs = []

    for line in str.strip().split('\n'):
        coeff_str, pauli_op = line.split('] ')
        coeff = complex(coeff_str.strip('['))  
        Coeffs.append(coeff)
        Pauli_Ops.append(pauli_op.strip())

    print("Pauli_Ops =", Pauli_Ops)
    print("Coeffs =", Coeffs)

    return Pauli_Ops, Coeffs

In [4]:
with open("ZnO.txt", "r") as file:
    Ham_str = file.read()

Pauli_Ops, Coeffs = string_to_ops(Ham_str)

Pauli_Ops = ['XZXIIXZX', 'YIZZZZYI', 'IYZYYZYI', 'XZXIYZYI', 'XZZYYZZX', 'XZZIZZXI', 'XZYIYZXI', 'YZYYZZZY', 'IIZIZIII', 'IIIIYXXY', 'ZIXZXIII', 'IIIZIIZI', 'XYIIIIYX', 'YXIIIIXY', 'YZZZZZYZ', 'YZYXZZZX', 'IIYZZYXX', 'YXXZZYII', 'XZZZIZXI', 'XZZXIXXI', 'XXIIYYII', 'XZXIXZXI', 'IYZZZZZY', 'XYIIYXII', 'IIIYYIYY', 'XZXIIYZY', 'IIZIIIZI', 'IIZIIIIZ', 'XZZYIYXI', 'IIIZIIIZ', 'IIXYIIYX', 'IIYZYIII', 'IYXIIXYI', 'IIIXZXZI', 'XZZZZIXI', 'IIXXYYII', 'YYIIIIXX', 'YZZZYXZX', 'YZYIXZXI', 'IYIZZZZY', 'IXXYZZYI', 'XZXYZZZY', 'IXZXYZYI', 'IYXXZZYI', 'IXZIZZZX', 'IYXIXZZY', 'IIZZIIII', 'IIIIZIZI', 'XYYXIIII', 'IXXIIYYI', 'IYZYIYZY', 'IZIZIIII', 'XZZXYZZY', 'IXYIYZZX', 'IYYIYZZY', 'IXIZZZZX', 'IYZIZZZY', 'XZZZXXZX', 'IXZZXYYI', 'ZIIIIIIZ', 'YZZIZZYI', 'IIIYXIXY', 'IIIIIIIZ', 'IXZXIYZY', 'IYZZZZIY', 'YXIIXYII', 'IXZXXZXI', 'IXZZYYXI', 'IZYZYIII', 'ZIIIIZII', 'IZIYZYII', 'ZIIYZYII', 'IIXZZXYY', 'IIXZZYYX', 'IIXZXIZI', 'IXXIXZZX', 'IZIXZXII', 'YZZZZIYI', 'IIIXZXIZ', 'IYZZYXXI', 'IIIXXIXX', 'YZZZIZYI', 'II

In [ ]:
# Adam optimizer hyperparameters
learning_rate = 0.01
beta1 = 0.9
beta2 = 0.999
epsilon = 1e-8

In [ ]:
file = "Data/"

In [ ]:
def plot_log(E_hist, E_err_hist):
    #ya, iter = smpl.median_filter(np.real(E_hist))
    ya = np.real(E_hist)
    xa = np.arange(len(ya))
    ci = np.array(E_err_hist) #[iter]
    fig, ax = plt.subplots()
    ax.plot(ya)
    ax.fill_between(xa, (ya-ci), (ya+ci), color='r', alpha=.5)
    ax.set_ylabel("Energy")
    ax.set_xlabel("Epochs")
    plt.hlines(lmbd[0],0,len(xa)-1,color='r',linestyles='dashed')
    plt.show()

In [ ]:
# #Calculating quantities
# def calc_quants(smpl,grad,H):
#   s = smpl.enum(N)
#   local_energies = smpl.local_energy(H,s)
#   kernels = smpl.kernel(s)
#   phi = smpl.prob(s)

#   derivative_op_diag = np.zeros((12,2**N),dtype=complex)
#   local_gradients = np.zeros((12,2**N),dtype=complex)
#   s_vec = np.reshape(s, (len(s),1,N))

#   da,db,dw,du,dd,dc = get_params(grad)
#   dw = np.reshape(dw,-1)

#   idx_list = np.array([np.argmin(np.abs(da)), np.argmax(np.abs(da)), N+np.argmin(np.abs(db)), N+np.argmax(np.abs(db)), N+M+np.argmin(np.abs(dw)), N+M+np.argmax(np.abs(dw))])
#   idx_list = np.concatenate((idx_list,idx_list+N+M+N*M+N*D+D+N*N))

#   for i in range(12):
#     idx = idx_list[i]
#     var, _, _ = smpl.map_idx_to_var(idx)
#     local_gradients[i] = smpl.local_gradient(H,s,idx)
#     derivative_op_diag[i] = smpl.derivative_operator(s,s_vec,idx)[:,0]


def save_data(X,smpl,g,seed,E,E_smpl,prob_dist,sample_list):
  with open(file+"X_N="+str(N)+"_g="+str(g)+"_seed="+str(seed)+".txt", "ab") as f:
    np.savetxt(f, np.reshape(X,(1,-1)))
  with open(file+"Poly_N="+str(N)+"_g="+str(g)+"_seed="+str(seed)+".txt", "ab") as f:
    np.savetxt(f, np.reshape(smpl.poly,(1,-1)))
  with open(file+"E_N="+str(N)+"_g="+str(g)+"_seed="+str(seed)+".txt", "ab") as f:
    np.savetxt(f, [E])
  with open(file+"E_smpl_N="+str(N)+"_g="+str(g)+"_seed="+str(seed)+".txt", "ab") as f:
    np.savetxt(f, [E_smpl])
  with open(file+"prob_dist_N="+str(N)+"_g="+str(g)+"_seed="+str(seed)+".txt", "ab") as f:
    np.savetxt(f, np.reshape(prob_dist,(1,-1)))
  with open(file+"prob_dist_N="+str(N)+"_g="+str(g)+"_seed="+str(seed)+".txt", "ab") as f:
    np.savetxt(f, np.reshape(sample_list,(1,-1)))


In [ ]:
def jac(rbm,H,prob_dist=[],exact=False):
  if exact == True:
    grad_exact = rbm.grad_exact(H)
    return grad_exact
    
  else:
    grad = rbm.grad_Sampling(H,prob_dist)
    return grad

In [ ]:
import tensorflow as tf

In [ ]:
sample_size = 20000

seed = 1

E_hist=[]
E_smpl_hist=[]

rbm = RBM_surrogate(N=N,M=M,seed=seed)
x = tf.Variable(rbm.X)

tm=time.time()

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=beta1, beta_2=beta2, epsilon=epsilon)

for epoch in range(40):
  rbm = RBM_surrogate(x.numpy(),N,M,D,beta=beta)
  prob_dist, sample_list = Sampling(rbm, sample_size, burn=sample_size//10)

  E = rbm.Energy_exact(H)
  E_smpl = rbm.Energy_sampling(H,prob_dist)
  print("Energy: ", E, "\t +/- \t", np.abs(E_smpl - E))

  grad = jac(rbm,H,prob_dist)

  optimizer.apply_gradients([(grad, x)])

  E_hist.append(E)
  E_smpl_hist.append(E_smpl)

print("\n\n\nEnergy: ",E,"\n Time",time.time()-tm, lmbd[0])

plot_log()